In [61]:
import pandas as pd # type: ignore
import os

In [62]:
path = "../datasets/"
archivo = "BRCA.csv"
df = pd.read_csv(os.path.join(path, archivo))
df.head()

,Patient_ID,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
0,TCGA-D8-A1XD,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,15-Jan-17,19-Jun-17,Alive
1,TCGA-EW-A1OX,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,26-Apr-17,09-Nov-18,Dead
2,TCGA-A8-A079,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,08-Sep-17,09-Jun-18,Alive
3,TCGA-D8-A1XR,56.0,FEMALE,0.345090,-0.21147,-0.19304,0.124270,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,25-Jan-17,12-Jul-17,Alive
4,TCGA-BH-A0BF,56.0,FEMALE,0.221550,1.90680,0.52045,-0.311990,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,06-May-17,27-Jun-19,Dead


In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Patient_ID          334 non-null    object 
 1   Age                 334 non-null    float64
 2   Gender              334 non-null    object 
 3   Protein1            334 non-null    float64
 4   Protein2            334 non-null    float64
 5   Protein3            334 non-null    float64
 6   Protein4            334 non-null    float64
 7   Tumour_Stage        334 non-null    object 
 8   Histology           334 non-null    object 
 9   ER status           334 non-null    object 
 10  PR status           334 non-null    object 
 11  HER2 status         334 non-null    object 
 12  Surgery_type        334 non-null    object 
 13  Date_of_Surgery     334 non-null    object 
 14  Date_of_Last_Visit  317 non-null    object 
 15  Patient_Status      321 non-null    object 
dtypes: float

In [64]:
df = df.drop(columns=['Date_of_Surgery', 'Date_of_Last_Visit'])

In [65]:
# Mostrar el número de valores nulos por columna
null_counts = df.isnull().sum()

# Filtrar solo las columnas que tienen al menos un valor nulo
null_columns = null_counts[null_counts > 0]

# Mostrar el resultado
print(null_columns)

Patient_ID         7
Age                7
Gender             7
Protein1           7
Protein2           7
Protein3           7
Protein4           7
Tumour_Stage       7
Histology          7
ER status          7
PR status          7
HER2 status        7
Surgery_type       7
Patient_Status    20
dtype: int64


In [66]:
# Seleccionamos las columnas que tienen 7 valores nulos
cols_with_7_na = df.columns[df.isnull().sum() == 7]

# Filtramos las filas que tienen al menos un NaN en esas columnas
rows_with_na = df[df[cols_with_7_na].isnull().any(axis=1)]

# Mostramos los índices de esas filas
print(rows_with_na.index)

# También puedes ver cuántas filas son
print(f"Número de filas con al menos un NaN en esas columnas: {len(rows_with_na)}")

Index([334, 335, 336, 337, 338, 339, 340], dtype='int64')
Número de filas con al menos un NaN en esas columnas: 7


In [67]:
print(len(rows_with_na))  # Debería imprimir 7 si todos los NaN están alineados

7


In [68]:
df = df.drop(rows_with_na.index)

In [69]:
# 1. Eliminar la columna Patient_ID  
df = df.drop(columns=['Patient_ID'])

# 2. Convertir todas las columnas de tipo object a category
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype('category')

# 3. Convertir la columna Age a int64
df['Age'] = df['Age'].fillna(df['Age']).astype('int64')

In [70]:
unseen_data = df[df['Patient_Status'].isna()]
data = df[~df['Patient_Status'].isna()]

print("Shape of Unseen Data: ", unseen_data.shape)
print("Shape of Data: ", data.shape)

Shape of Unseen Data:  (13, 13)
Shape of Data:  (321, 13)


# POSIBLEMNTE HAY QUE HACER BOOTSTRAP!

In [71]:
data['Patient_Status'].value_counts()

Patient_Status
Alive    255
Dead      66
Name: count, dtype: int64

# HABRÁ QUE APLICAR ALGUNA TÉCNICA COMO SMOTE PARA CLASES IMBALANCEADAS!

In [72]:
import mlflow
from mlflow.models import infer_signature
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [78]:
y = data['Patient_Status'].values
X = data.drop('Patient_Status', axis=1)

# apply one-hot-encoding
columnas_categoricas = X.select_dtypes(include='category').columns
dummies = pd.get_dummies(X[columnas_categoricas], drop_first=False)
X = pd.concat([X.drop(columns=columnas_categoricas), dummies], axis=1)

# split de data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# define the model hyperparameters --> Logistic Regression
params = {
    'penalty': None,
    'solver': 'lbfgs',
    'max_iter': 1000,
    'multi_class': 'auto',
    'random_state': 77
}

# train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000, penalty=None, random_state=77)

In [79]:
# Prediction on the test set
y_pred = lr.predict(X_test)
y_pred

array(['Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive',
       'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive',
       'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive',
       'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive',
       'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive',
       'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive',
       'Alive', 'Alive', 'Alive', 'Alive', 'Dead', 'Alive', 'Alive',
       'Alive', 'Alive', 'Dead', 'Alive', 'Alive', 'Alive', 'Alive',
       'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive', 'Alive',
       'Alive', 'Alive'], dtype=object)

In [80]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuarcy: ", accuracy)

Accuarcy:  0.7846153846153846


In [81]:
# set tracking uri
mlflow.set_tracking_uri(uri='http://127.0.0.1:5000')

# create a new MLflow experiment
mlflow.set_experiment('MLflow Experimental')

# start an MLflow run
with mlflow.start_run():
    # log the hyperparameters
    mlflow.log_params(params)

    # log the accuracy metrics
    mlflow.log_metric('accuracy', accuracy)

    # set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag('Training Info', 'Basic Logistic Regression for BRCA data')

    # infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))
    
    # log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path='brca_model',
        signature=signature,
        input_example=X_train,
        registered_model_name='tracking-mlflow-LR'
    )

2025/05/04 21:22:40 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Experimental' does not exist. Creating a new experiment.
c:\Users\Usuario\anaconda3\envs\health\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Successfully registered model 'tracking-mlflow-LR'.
2025/05/04 21:22:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-mlflow-LR, version 1


🏃 View run zealous-bug-707 at: http://127.0.0.1:5000/#/experiments/172291101050552261/runs/2a996c437e0244ae849119b908efb6d2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/172291101050552261


Created version '1' of model 'tracking-mlflow-LR'.
